In [1]:
###############################################
## Numpy/PyTorch implementation of Sichel Distribution
## See: https://github.com/cran/gamlss.dist
## See (count distributions - page197): http://www.gamlss.com/wp-content/uploads/2013/01/gamlss-manual.pdf 
##
## Author: Chris Meaney
## Date: August 2021
###############################################

In [2]:
## Dependency modules
import numpy as np
import pandas as pd
import torch
from scipy.special import gammaln
from scipy.special import kv
from sinfo import sinfo

In [3]:
##########################################################
## Use pandas to import data, and store as DataFrame
## Data are 1) response/target variable (number of fish = count random variable), 2) lake size (single continous feature/predictor)
##########################################################
dat = pd.read_csv('C://Users//ChristopherMeaney//Desktop//PyTorch_Stuff//pytorch_count_dists//species.csv', encoding='latin1')
dat.head(n=15)

,fish,lake,x,scale_x
0,10,5,1.609438,-1.533430
1,37,41,3.713572,-0.901903
2,60,171,5.141664,-0.473281
3,113,25719,10.154985,1.031399
4,99,59596,10.995344,1.283621
5,13,1,0.000000,-2.016481
6,30,44,3.784190,-0.880708
7,114,58016,10.968474,1.275556
8,112,19477,9.876990,0.947962
9,17,10,2.302585,-1.325392


In [4]:
## Describe the data
dat.fish.describe()

count     70.000000
mean      41.742857
std       47.849609
min        5.000000
25%       14.000000
50%       21.500000
75%       47.500000
max      245.000000
Name: fish, dtype: float64

In [5]:
mu_ = dat.fish.mean()
mu_

41.74285714285714

In [6]:
sigma_ = np.sqrt(dat.fish.var())
sigma_

47.84960912241293

In [7]:
###################################################
## Numpy implementation of Sichel Loss/Density Function
## Function is basically a Numpy implementation of Rigby et al gamlss.dist R Code (which calls tofySICHEL2.c)
## https://github.com/cran/gamlss.dist/tree/master/src/tofySICHEL2.c
###################################################
def d_SICHEL_np(x, mu, sigma, nu, log=True):
    ## Determine length of data vector and parameters 
    ly = np.max(np.array([len(x), len(mu), len(sigma), len(nu)]))  
    #x = np.repeat(a=x, repeats=ly)      
    nsigma = np.repeat(a=sigma, repeats=ly)
    nmu = np.repeat(a=mu, repeats=ly)
    nnu = np.repeat(a=nu, repeats=ly)
    ## Global constants in SICHEL density     
    cvec = np.exp(np.log(kv(nnu + 1, (1/nsigma))) - np.log(kv(nnu, (1/nsigma))))
    alpha = np.sqrt(1 + 2*nsigma * (nmu/cvec))/nsigma
    lbes = np.log(kv(nnu + 1, alpha)) - np.log(kv(nnu, alpha))
    ## Initial vectors to store computed SICHEL density values
    ny = int(len(x))
    maxyp1 = np.max(x) + 1
    tofY = np.zeros(shape=(maxyp1))
    sumlty = np.zeros(shape=(ly))
    ## Big for loop to compute SICHEL density (or log-density)
    ## This is directly from Rigby et al: tofySICHEL2.c code.
    for i in range(1, ny+1):
        iy = x[i-1] + 1
        tofY[0] = (nmu[i-1]/cvec[i-1]) * np.power(1 + 2*nsigma[i-1]*(nmu[i-1]/cvec[i-1]), -1/2) * np.exp(lbes[i-1]) 
        alpha = np.sqrt(1 + 2*nsigma[i-1]*(nmu[i-1]/cvec[i-1]))/nsigma[i-1]
        sumT = 0 
        ## Start inner loop to compute rest of SICHEL density
        if (x[i-1]==0):
            sumT = 0
        else:
            for j in range(1, iy):
                tofY[j] = ( (cvec[i-1]*nsigma[i-1] * (2*(j+nnu[i-1])/nmu[i-1])) + (1/tofY[j-1])) * np.power(nmu[i-1]/(nsigma[i-1]*alpha*cvec[i-1]), 2)
                sumT = sumT + np.log(tofY[j-1])
        sumlty[i-1] = sumT
    ## Add the kernel of the SICHEL density back to other constant component
    logfy = -gammaln(x+1) - nnu*np.log(nsigma * alpha) + sumlty + np.log(kv(nnu, alpha)) - np.log(kv(nnu, (1/nsigma)))
    ## Log={T,F} flag: T=return log-density; F=return density
    if(log==False):
        fy = np.exp(logfy)  
    else:
        fy = logfy
    ## If sigma>10000 (large) AND nu>0 then can approx. SICHEL using NBI
    # fy <- ifelse((nsigma>10000)&(nnu>0), dnbinom(x, size=abs(nnu), mu=nmu, log=log), fy)
    ## Return log-density (or density) to user
    return fy

d_SICHEL_np(x=np.arange(10), mu=np.array([1]), sigma=np.array([1]), nu=np.array([-0.5]), log=True)

array([-0.73205081, -1.28135695, -2.06806388, -2.85477081, -3.59360409,
       -4.28363973, -4.93343401, -5.55145683, -6.14437891, -6.71720088])

In [8]:
## WARNING: below is NOT a pure PyTorch implementation of SICHEL density/distribution
## ISSUE: PyTorch and torch.special does NOT have implementation of BESSEL functions
## WORKAROUND: we roll a PyTorch implementation where possible; however, default back to scipy.special.kv() where needed
## ISSUE: since it is NOT a pure PyTorch implementation we CANNOT run AutoGrad/BackProp/etc. on this loss (due to detach of values/nodes from computational graph)
## WORKAROUND: I am going to **try** to roll something like torch.special.kv()...will need to test/go-slow (since limitted experience with this)

In [9]:
###################################################
## PyTorch implementation of SICHEL Loss/Density Function
## Function is basically a Numpy implementation of Rigby et al gamlss.dist R Code (which calls tofySICHEL2.c)
## https://github.com/cran/gamlss.dist/tree/master/src/tofySICHEL2.c
###################################################
def d_PIG_th(x, mu, sigma, nu, log=True): 
    ## Determine length of data vector and parameters 
    ly = int(torch.max(torch.Tensor([len(x), len(mu), len(sigma)])).item())
    #x = np.repeat(a=x, repeats=ly)      
    nsigma = sigma.repeat(ly)
    nmu = mu.repeat(ly)
    nnu = nu.repeat(ly)
    ## Global constants in SICHEL density     
    cvec = torch.exp(torch.log(kv(nnu + 1, (1/nsigma))) - torch.log(kv(nnu, (1/nsigma))))
    alpha = torch.sqrt(1 + 2*nsigma * (nmu/cvec))/nsigma
    lbes = torch.log(kv(nnu + 1, alpha)) - torch.log(kv(nnu, alpha))
    ## Initial vectors to store computed PIG density values
    ny = int(len(x))
    maxyp1 = torch.max(x) + 1
    tofY = torch.zeros(maxyp1)
    sumlty = torch.zeros(ly)
    ## Big for loop to compute SICHEL density (or log-density)
    ## This is directly from Rigby et al: tofySICHEL2.c code.
    for i in torch.arange(1, ny+1):
        iy = x[i.item()-1] + 1
        tofY[0] = (nmu[i.item()-1]/cvec[i.item()-1]) * torch.pow(1 + 2*nsigma[i.item()-1]*(nmu[i.item()-1]/cvec[i.item()-1]), -1/2) * torch.exp(lbes[i.item()-1]) 
        alpha = torch.sqrt(1 + 2*nsigma[i.item()-1]*(nmu[i.item()-1]/cvec[i.item()-1]))/nsigma[i.item()-1]
        sumT = 0 
        ## Start inner loop to compute rest of SICHEL density
        if (x[i.item()-1]==0):
            sumT = 0
        else:
            for j in torch.arange(1, iy):
                tofY[j.item()] = ( (cvec[i.item()-1]*nsigma[i.item()-1] * (2*(j.item()+nnu[i.item()-1])/nmu[i.item()-1])) + (1/tofY[j.item()-1])) * torch.pow(nmu[i.item()-1]/(nsigma[i.item()-1]*alpha*cvec[i.item()-1]), 2)
                sumT = sumT + torch.log(tofY[j.item()-1])
        sumlty[i.item()-1] = sumT
    ## Add the kernel of the SICHEL density back to other constant component
    logfy = -torch.lgamma(x+1) - nnu*torch.log(nsigma * alpha) + sumlty + torch.log(kv(nnu, alpha)) - torch.log(kv(nnu, (1/nsigma)))
    ## Log={T,F} flag: T=return log-density; F=return density
    if(log==False):
        fy = torch.exp(logfy)  
    else:
        fy = logfy
    ## If sigma>10000 (large) AND nu>0 then can approx. SICHEL using NBI
    # fy <- ifelse((nsigma>10000)&(nnu>0), dnbinom(x, size=abs(nnu), mu=nmu, log=log), fy)
    ## Return log-density (or density) to user
    return fy

d_PIG_th(x=torch.arange(10), mu=torch.Tensor([1]), sigma=torch.Tensor([1]), nu=torch.Tensor([-0.5]), log=True)

tensor([-0.7321, -1.2814, -2.0681, -2.8548, -3.5936, -4.2836, -4.9334, -5.5515,
        -6.1444, -6.7172])

In [10]:
## WARNING: read me... 
## Below is user-defined R implementation of SICHEL density
## We also compare against gamlss.dist::dSICHEL() code rolled out in gamlss.dist package
## Documentation on page 197: http://www.gamlss.com/wp-content/uploads/2013/01/gamlss-manual.pdf
## You will see that above Numpy and PyTorch implementations agree with R implementation up to many decimal places

In [11]:
'''
> 
> library(gamlss.dist)
> 
> d_SICHEL<-function(x, mu=1, sigma=1, nu=-0.5, log=FALSE) { 
+     ## Warning messages on paramter and data space constraint violations
+     if (any(mu <= 0) )  stop(paste("mu must be greater than 0 ", "\n", "")) 
+     if (any(sigma <= 0) )  stop(paste("sigma must be greater than 0 ", "\n", "")) 
+     if (any(x < 0) )  stop(paste("x must be >=0", "\n", ""))  
+     ## Determine length of data vector and parameters 
+     ly <- max(length(x),length(mu),length(sigma),length(nu))
+     x <- rep(x, length = ly)      
+     nsigma <- rep(sigma, length = ly)
+     nmu <- rep(mu, length = ly)   
+     nnu <- rep(nu, length = ly)
+     ## Global constants in SICHEL density     
+     cvec <- exp(log(besselK((1/nsigma), nnu + 1)) - log(besselK((1/nsigma), nnu)))
+     alpha <- sqrt(1 + 2*nsigma * (nmu/cvec))/nsigma
+     lbes <- log(besselK(alpha, nnu + 1)) - log(besselK((alpha), nnu))
+     ## Initial vectors to store computed PIG density values
+     ny <- as.integer(length(x))
+     maxyp1 <- max(x) + 1
+     tofY <- rep(NA_real_, maxyp1)
+     sumlty <- rep(NA_real_, ly)
+     ## Big for loop to compute SICHEL density (or log-density)
+     ## This is directly from Rigby et al: tofySICHEL2.c code.
+     ## I **think** it looks like its implementing recursive mixed-Pois prob calc
+     ## This is likely why (for large vectors, with large counts) that is done in C
+     for (i in 1:ny) {
+         iy <- x[i] + 1
+         tofY[1] <- (nmu[i]/cvec[i]) * (1 + 2*nsigma[i]*(nmu[i]/cvec[i]))^(-1/2) * exp(lbes[i]) 
+         alpha = sqrt(1 + 2*nsigma[i]*(nmu[i]/cvec[i]))/nsigma[i]
+         sumT <- 0 
+         ## Start inner loop to compute rest of PIG density
+         if (x[i]==0) {
+             sumT <- 0
+         } else {
+             for (j in 1:(iy-1)) {
+                 tofY[j + 1] <- ( (cvec[i]*nsigma[i] * (2*(j+nnu[i])/nmu[i])) + (1/tofY[j])) * (nmu[i]/(nsigma[i]*alpha*cvec[i]))^2
+                 sumT <- sumT + log(tofY[j])
+             }
+         }
+         sumlty[i] <- sumT
+     }
+     ## Add the kernel of the PIG density back to other constant component
+     logfy <- -lgamma(x+1) - nnu*log(nsigma * alpha) + sumlty + log(besselK(alpha, nnu)) - log(besselK((1/nsigma), nnu))
+     ## Log={T,F} flag: T=return log-density; F=return density
+     if(log==FALSE) {
+         fy <- exp(logfy)  
+     } else {
+         fy <- logfy
+     }
+     ## If sigma>10000 (large) AND nu>0 then can approx. SICHEL using NBI
+     fy <- ifelse((nsigma>10000)&(nnu>0), dnbinom(x, size=abs(nnu), mu=nmu, log=log), fy)
+     ## Return log-density (or density) to user
+     return(fy)
+ }
 
> d_SICHEL(x=0:9, mu=1, sigma=1, nu=-0.5, log=TRUE)
 [1] -0.7320508 -1.2813570 -2.0680639 -2.8547708 -3.5936041 -4.2836397 -4.9334340 -5.5514568 -6.1443789 -6.7172009

> dSICHEL(x=0:9, mu=1, sigma=1, nu=-0.5, log=TRUE)
 [1] -0.7320508 -1.2813570 -2.0680639 -2.8547708 -3.5936041 -4.2836397 -4.9334340 -5.5514568 -6.1443789 -6.7172009
> 
'''
;

''

In [12]:
##############################################
## SICHEL Model - try to learn MLE of fish count data; via AutoGrad/SGD implementation in PyTorch
############################################## 

In [13]:
## Instantiate data tensor, and variable for (binomial) model parameters
x = torch.autograd.Variable(torch.from_numpy(dat.fish.to_numpy())).type(torch.FloatTensor)
l_mu = torch.autograd.Variable(torch.rand(1), requires_grad=True)
l_sigma = torch.autograd.Variable(torch.rand(1), requires_grad=True) 
l_nu = torch.autograd.Variable(torch.rand(1), requires_grad=True) 

In [14]:
def sichel_nll(x, mu, sigma, nu): 
    ## Determine length of data vector and parameters 
    ly = int(torch.max(torch.Tensor([len(x), len(mu), len(sigma)])).item())
    #x = np.repeat(a=x, repeats=ly)      
    nsigma = sigma.repeat(ly)
    nmu = mu.repeat(ly)
    nnu = nu.repeat(ly)
    ## Global constants in SICHEL density     
    cvec = torch.exp(torch.log(kv(nnu + 1, (1/nsigma))) - torch.log(kv(nnu, (1/nsigma))))
    alpha = torch.sqrt(1 + 2*nsigma * (nmu/cvec))/nsigma
    lbes = torch.log(kv(nnu + 1, alpha)) - torch.log(kv(nnu, alpha))
    ## Initial vectors to store computed PIG density values
    ny = int(len(x))
    maxyp1 = torch.max(x) + 1
    tofY = torch.zeros(maxyp1)
    sumlty = torch.zeros(ly)
    ## Big for loop to compute SICHEL density (or log-density)
    ## This is directly from Rigby et al: tofySICHEL2.c code.
    for i in torch.arange(1, ny+1):
        iy = x[i.item()-1] + 1
        tofY[0] = (nmu[i.item()-1]/cvec[i.item()-1]) * torch.pow(1 + 2*nsigma[i.item()-1]*(nmu[i.item()-1]/cvec[i.item()-1]), -1/2) * torch.exp(lbes[i.item()-1]) 
        alpha = torch.sqrt(1 + 2*nsigma[i.item()-1]*(nmu[i.item()-1]/cvec[i.item()-1]))/nsigma[i.item()-1]
        sumT = 0 
        ## Start inner loop to compute rest of SICHEL density
        if (x[i.item()-1]==0):
            sumT = 0
        else:
            for j in torch.arange(1, iy):
                tofY[j.item()] = ( (cvec[i.item()-1]*nsigma[i.item()-1] * (2*(j.item()+nnu[i.item()-1])/nmu[i.item()-1])) + (1/tofY[j.item()-1])) * torch.pow(nmu[i.item()-1]/(nsigma[i.item()-1]*alpha*cvec[i.item()-1]), 2)
                sumT = sumT + torch.log(tofY[j.item()-1])
        sumlty[i.item()-1] = sumT
    ## Add the kernel of the SICHEL density back to other constant component
    logfy = -torch.lgamma(x+1) - nnu*torch.log(nsigma * alpha) + sumlty + torch.log(kv(nnu, alpha)) - torch.log(kv(nnu, (1/nsigma)))
    ## Return neg log lik to user
    nll = -torch.sum(logfy)
    return nll

In [15]:
torch.autograd.set_detect_anomaly(True)

## Learning rate
learning_rate_mu = 2e-5
learning_rate_sigma = 2e-5
learning_rate_nu = 2e-5

## Training loop
for t in range(25000):
    ## Backprop on negative log likelihood loss
    NLLsichel = sichel_nll(x=x, mu=l_mu, sigma=l_sigma, nu=l_nu) 
    NLLsichel.backward()
    ## Logging to console
    if t % 1000 == 0:
        print("Iteration = ", t, 
              "loglik  =", NLLpig.data.numpy(), 
              "lmu =", l_mu.data.numpy(), 
              "lsigma =", l_sigma.data.numpy(),
              "lnu =", l_nu.data.numpy(),
              "dL/dlmu = ", l_mu.grad.data.numpy(), 
              "dL/dlsigma = ", l_sigma.grad.data.numpy(),
              "dL/dlnu = ", l_nu.grad.data.numpy())
    ## SGD update of parms
    l_mu.data -= learning_rate_mu * l_mu.grad.data
    l_sigma.data -= learning_rate_sigma * l_sigma.grad.data
    l_nu.data -= learning_rate_nu * l_nu.grad.data
    ## Zero the gradients
    l_mu.grad.data.zero_()
    l_sigma.grad.data.zero_()
    l_nu.grad.data.zero_()
    

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
########################
## Print session info to console 
########################
sinfo()